In [1]:
import pyspark
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 17:37:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("p_id", IntegerType(), True) # Assuming p_id is also an integer, if it's a string replace IntegerType() with StringType()
])
data = [
    (1, None),
    (2, 1),
    (3, 1),
    (4, 2),
    (5, 2)
]
df = spark.createDataFrame(data, schema)
df.show()

+---+----+
| id|p_id|
+---+----+
|  1|NULL|
|  2|   1|
|  3|   1|
|  4|   2|
|  5|   2|
+---+----+



In [4]:
df.createOrReplaceTempView("Tree")

23/11/15 17:37:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
result = spark.sql(
    """
    SELECT id, 
        CASE 
            WHEN p_id IS NULL 
                THEN 'Root'
            WHEN id IN (SELECT p_id FROM Tree WHERE p_id IS NOT NULL)
                THEN 'Inner'
            ELSE 'Leaf' END AS type
    FROM Tree
    """
)
result.show()

+---+-----+
| id| type|
+---+-----+
|  1| Root|
|  2|Inner|
|  3| Leaf|
|  4| Leaf|
|  5| Leaf|
+---+-----+



In [8]:
result = spark.sql(
    """
    SELECT id, 'Root' AS TYPE FROM Tree WHERE p_id IS NULL 
    UNION
    SELECT id, 'Inner' AS TYPE FROM Tree 
        WHERE p_id IS NOT NULL AND id IN (SELECT DISTINCT p_id FROM Tree WHERE p_id IS NOT NULL)
    UNION
    SELECT id, 'Leaf' AS TYPE FROM Tree 
        WHERE p_id IS NOT NULL AND id NOT IN (SELECT DISTINCT p_id FROM Tree WHERE p_id IS NOT NULL)
    """
)
result.show()

+---+-----+
| id| TYPE|
+---+-----+
|  1| Root|
|  2|Inner|
|  3| Leaf|
|  4| Leaf|
|  5| Leaf|
+---+-----+

